<a href="https://colab.research.google.com/github/venezianof/booksum/blob/main/module-6/assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assistants

[Assistants](https://langchain-ai.github.io/langgraph/concepts/assistants/#resources) give developers a quick and easy way to modify and version agents for experimentation.

## Supplying configuration to the graph

Our `task_maistro` graph is already set up to use assistants!

It has a `configuration.py` file defined and loaded in the graph.

We access configurable fields (`user_id`, `todo_category`, `task_maistro_role`) inside the graph nodes.

## Creating assistants

Now, what is a practical use-case for assistants with the `task_maistro` app that we've been building?

For me, it's the ability to have separate ToDo lists for different categories of tasks.

For example, I want one assistant for my personal tasks and another for my work tasks.

These are easily configurable using the `todo_category` and `task_maistro_role` configurable fields.

![Screenshot 2024-11-18 at 9.35.55 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/673d50597f4e9eae9abf4869_Screenshot%202024-11-19%20at%206.57.01%E2%80%AFPM.png)

In [26]:
%%capture --no-stderr
%pip install -U langgraph_sdk

This is the default assistant that we created when we deployed the graph.

In [27]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

### Personal assistant

This is the personal assistant that I'll use to manage my personal tasks.

In [28]:
personal_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro",
    config={"configurable": {"todo_category": "personal"}}
)
print(personal_assistant)

ConnectError: All connection attempts failed

In [21]:
import os
    url_for_cli_deployment = os.environ.get("LANGGRAPH_URL", "your_cloud_url")

IndentationError: unexpected indent (<ipython-input-21-de48d4d1bb17>, line 2)

In [20]:
import os
# If using a cloud deployment, uncomment the following line and replace with your cloud URL
url_for_cli_deployment = os.environ.get("LANGGRAPH_URL", "your_cloud_url")
# url_for_cli_deployment = "http://localhost:8123"  # For local deployments

# Print the URL for verification
print(f"Connecting to: {url_for_cli_deployment}")

client = await get_client(url=url_for_cli_deployment)

# ... rest of your code

# Deleting an assistant (example for work_assistant)
await client.assistants.delete(work_assistant_id)

Connecting to: your_cloud_url


TypeError: object LangGraphClient can't be used in 'await' expression

In [22]:
import os
# If using a cloud deployment, uncomment the following line and replace with your cloud URL
url_for_cli_deployment = os.environ.get("LANGGRAPH_URL", "your_cloud_url")
# url_for_cli_deployment = "http://localhost:8123"  # For local deployments

# Print the URL for verification
print(f"Connecting to: {url_for_cli_deployment}")

client = await get_client(url=url_for_cli_deployment)

# ... rest of your code

# Deleting an assistant (example for work_assistant)
await client.assistants.delete(work_assistant_id)

Connecting to: your_cloud_url


TypeError: object LangGraphClient can't be used in 'await' expression

In [ ]:
import os

# If using a cloud deployment, uncomment the following line and replace with your cloud URL
# url_for_cli_deployment = os.environ.get("LANGGRAPH_URL", "your_cloud_url")
url_for_cli_deployment = "http://localhost:8123"  # For local deployments

# Add this line to print the URL for verification
print(f"Connecting to: {url_for_cli_deployment}")

client = get_client(url=url_for_cli_deployment)

# ... rest of your code

Let's update this assistant to include my `user_id` for convenience, [creating a new version of it](https://langchain-ai.github.io/langgraph/cloud/how-tos/assistant_versioning/#create-a-new-version-for-your-assistant).

In [23]:
task_maistro_role = """You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:

- Help track and organize personal tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- Proactively ask for deadlines when new tasks are added without them
- Maintain a supportive tone while helping the user stay accountable
- Help prioritize tasks based on deadlines and importance

Your communication style should be encouraging and helpful, never judgmental.

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Would you like to add one to help us track it better?"""

configurations = {"todo_category": "personal",
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"],
    config={"configurable": configurations}
)
print(personal_assistant)

NameError: name 'personal_assistant' is not defined

### Work assistant

Now, let's create a work assistant. I'll use this for my work tasks.

In [ ]:
task_maistro_role = """You are a focused and efficient work task assistant.

Your main focus is helping users manage their work commitments with realistic timeframes.

Specifically:

- Help track and organize work tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:
  • Developer Relations features: typically 1 day
  • Course lesson reviews/feedback: typically 2 days
  • Documentation sprints: typically 3 days
- Help prioritize tasks based on deadlines and team dependencies
- Maintain a professional tone while helping the user stay accountable

Your communication style should be supportive but practical.

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Based on similar tasks, this might take [suggested timeframe]. Would you like to set a deadline with this in mind?"""

configurations = {"todo_category": "work",
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

work_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro",
    config={"configurable": configurations}
)
print(work_assistant)

In [ ]:
!curl http://localhost:8123/healthz

## Using assistants

Assistants will be saved to `Postgres` in our deployment.  

This allows us to easily search [search](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/) for assistants with the SDK.

In [ ]:
assistants = await client.assistants.search()
for assistant in assistants:
    print({
        'assistant_id': assistant['assistant_id'],
        'version': assistant['version'],
        'config': assistant['config']
    })

We can manage them easily with the SDK. For example, we can delete assistants that we're no longer using.

In [24]:
await client.assistants.delete("assistant_id")

ConnectError: All connection attempts failed

In [ ]:
print(f"Connecting to: {url_for_cli_deployment}")
   client = get_client(url=url_for_cli_deployment)

Let's set the assistant IDs for the `personal` and `work` assistants that I'll work with.

In [ ]:
work_assistant_id = assistants[0]['assistant_id']
personal_assistant_id = assistants[1]['assistant_id']

### Work assistant

Let's add some ToDos for my work assistant.

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

user_input = "Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"],
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

In [ ]:
user_input = "Create another ToDo: Finalize set of report generation tutorials."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"],
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

The assistant uses it's instructions to push back with task creation!

It asks me to specify a deadline :)

In [ ]:
user_input = "OK, for this task let's get it done by next Tuesday."
async for chunk in client.runs.stream(thread["thread_id"],
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

### Personal assistant

Similarly, we can add ToDos for my personal assistant.

In [25]:
user_input = "Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"],
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

ConnectError: All connection attempts failed

In [ ]:
user_input = "Give me a todo summary."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"],
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()